<a href="https://colab.research.google.com/github/NacliNaclo/EDOS/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# file_num = 000000
# a =[0]*197
# for i in range(0,count):
#   image_path = '/content/drive/MyDrive/medical_robotics/v1/v1/frame_rgb/'+ str(file_num + i).zfill(6)+'.png'
#   # Load the image without resizing or preprocessing
#   image_1 = tf.io.read_file(image_path)
#   image_1= tf.image.decode_image(image_1, channels=3)
#   a[i] = image_1

In [ ]:
# dataset = np.stack(a, axis=0)

In [ ]:
# np.save('/content/drive/MyDrive/medical_robotics/dataset.npy', dataset)

# DATASET

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

In [6]:
loaded_dataset = np.load('/content/drive/MyDrive/medical_robotics/dataset.npy')

In [8]:
loaded_dataset.shape #so now we have created our dataset in the correct shape

(197, 1024, 1280, 3)

In [20]:
start_images = loaded_dataset[0:196,:,:,:] #the last image has no other correlated images in order to compute the homogenous trasnformation matrix

In [21]:
start_images.shape

(196, 1024, 1280, 3)

In [23]:
final_images = loaded_dataset[1:197,:,:,:] #the last index is exluded (so if we write 0:1 it extracts only 1 value, that in position 0)

In [24]:
final_images.shape

(196, 1024, 1280, 3)

In [ ]:
# so now we have ur 2 dataset that will be input to our 2 network

# TARGET

In [37]:
import json

In [55]:
def tranformation_matrix_extrac(folder_path):
  # looking for number of files in the folder
  import os
  count = 0
  # Iterate directory
  for path in os.listdir(folder_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder_path, path)):
        count += 1

  file_num = 000000
  tranf_matrices = [0]*(count-1)
  for i in range(0,count-1): #the upper boundar is exclude
    j = i + 1
    source = folder_path + '/frame_data' + str(file_num + i).zfill(6) + '.json'
    end = folder_path + '/frame_data' + str(file_num + j).zfill(6) + '.json'

    f_source = open(source)
    data_source = json.load(f_source)
    T_0 = np.array(data_source['camera-pose'])

    f_end = open(end)
    data_end = json.load(f_end)
    T_1 = np.array(data_end['camera-pose'])

    tranf_matrices[i] = np.matmul(np.linalg.inv(T_0),T_1)

  return tranf_matrices



In [56]:
transf_matrices = tranformation_matrix_extrac('/content/drive/MyDrive/medical_robotics/v1/v1/frame_data')

In [57]:
targets = np.stack(transf_matrices, axis=0)

In [76]:
targets.shape

(196, 4, 4)

In [69]:
targets[0]

array([[ 9.99996758e-01, -2.66848080e-03, -1.38534687e-04,
        -1.76657959e-02],
       [ 2.66837263e-03,  9.99996429e-01, -8.90276436e-04,
        -1.13811995e-01],
       [ 1.40861180e-04,  8.89828332e-04,  9.99999821e-01,
         2.77779557e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [70]:
targets[0,0:3,3]

array([-0.0176658 , -0.11381199,  0.27777956])

## POSITION

In [66]:
positions = [0]*196

In [67]:
for i in range(0,196):
  positions[i] = targets[i,0:3,3]

In [71]:
positions = np.stack(positions, axis=0)  #in order to move from an list to an array

In [90]:
positions.shape

(196, 3)

## ORIENTATION

In [80]:
def matrix_to_rpy(R): # i think that this type of representation is not the correct one, since we have some ambiguity (the quaternians are better)
    pitch = np.arcsin(-R[2, 0])
    roll = np.arctan2(R[2, 1], R[2, 2])
    yaw = np.arctan2(R[1, 0], R[0, 0])
    return [roll, pitch, yaw]

In [82]:
angles = [0]*196
for i in range(0,196):
  angles[i] = matrix_to_rpy(targets[i,:,:])

In [86]:
angles = np.stack(angles, axis=0)

In [87]:
angles.shape

(196, 3)

# MODEL

In [92]:
def build_model(input_shape_1, input_shape_2):
    tf.random.set_seed(seed)

    # Build the neural network layer by layer (1st network)
    input_layer_1 = tfkl.Input(shape=input_shape_1, name='Input')

    conv1_1 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(input_layer_1)
    pool1_1 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv1_1)

    conv2_1 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool1_1)
    pool2_1 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv2_1)

    conv3_1 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool2_1)
    pool3_1 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv3_1)

    flattening_layer_1 = tfkl.Flatten(name='Flatten')(pool3_1)

#############################################################################

    #2nd network
    input_layer_2 = tfkl.Input(shape=input_shape_2, name='Input')

    conv1_2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(input_layer_2)
    pool1_2 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv1_2)

    conv2_2 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool1_2)
    pool2_2 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv2_2)

    conv3_2 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool2_2)
    pool3_2 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv3_2)

    flattening_layer_2 = tfkl.Flatten(name='Flatten')(pool3_2)


############################################################################

    # now we need to merge the due flattering layer from the 2 network and at
    # then input it inside the classification model

    merged = Concatenate()([flattening_layer_1,flattening_layer_2])


###########################################################################


    # classifation layer

    classifier_layer = tfkl.Dense(
        units=128,
        name='Classifier',
        activation='relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(merged)

    classifier_layer = tfkl.Dropout(0.5, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(
        units=6,
        activation='softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name='Output'
    )(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model